<a href="https://colab.research.google.com/github/BigAlEE/BigAlEE/blob/main/Intro_to__sql_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# We are going to explore setting up a SQLite database to learn in!

The following is an Entity-Relationship Diagram (ERD) of the database we will be creating using Python. This ERD acts as a blueprint for our database structure, providing a clear visualization of the relationships between entities. It serves as a useful reference throughout the development process, helping us identify data types, primary keys, and foreign key relationships. Additionally, it will assist in maintaining data integrity and ensuring that the database is structured efficiently. By referring to this diagram, we can easily track how different tables interact with each other and ensure that our database design supports the application’s needs effectively.

[ERD Diagram](https://www.lucidchart.com/pages/er-diagrams)

## Employees Table

| **Column Name**  | **Data Type** | **Constraints**    | **Description**                                 |
|------------------|---------------|--------------------|-------------------------------------------------|
| id               | INTEGER       | PRIMARY KEY        | Unique identifier for each employee             |
| name             | TEXT          | NOT NULL           | Name of the employee                            |
| department_id    | INTEGER       | NOT NULL           | Foreign key referencing departments table       |
| hire_date        | TEXT          | NOT NULL           | The date the employee was hired                 |

## Departments Table

| **Column Name**  | **Data Type** | **Constraints**    | **Description**                                 |
|------------------|---------------|--------------------|-------------------------------------------------|
| department_id               | INTEGER       | PRIMARY KEY        | Unique identifier for each department           |
| name             | TEXT          | NOT NULL           | Name of the department                          |

<br>

---

<br>

### Here is what our tables will look like:

**Employee data**

|  |  id | name    |   department_id | hire_date   |
|---:|-----:|:--------|----------------:|:------------|
|  0 |    1 | Alice   |               1 | 2020-01-15  |
|  1 |    2 | Bob     |               2 | 2019-07-22  |
|  2 |    3 | Charlie |               1 | 2021-03-30  |
|  3 |    4 | David   |               3 | 2018-06-12  |
|  4 |    5 | Eve     |               2 | 2022-11-03  |


<br>

**Department data**

|    |   department_id | name        |
|---:|----------------:|:------------|
|  0 |               1 | HR          |
|  1 |               2 | Engineering |
|  2 |               3 | Sales       |

In [ ]:
import sqlite3
import pandas as pd
import tabulate

# Setting up our data

We will be making 2 dictionaries and converting them to DataFrames

In [ ]:
# Create sample data for employees
employees_data = {
    "id": [1, 2, 3, 4, 5],
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "department_id": [1, 2, 1, 3, 2],
    "hire_date": ["2020-01-15", "2019-07-22", "2021-03-30", "2018-06-12", "2022-11-03"],
}

# Create sample data for departments
departments_data = {
    "department_id": [1, 2, 3],
    "name": ["HR", "Engineering", "Sales"],
}

# Create DataFrames
employees_df = pd.DataFrame(employees_data)
departments_df = pd.DataFrame(departments_data)



In this step, we will use Python to create a local database directly from our DataFrames. This approach offers a straightforward and efficient way to set up a database, especially when compared to the more manual process of configuring it within a traditional SQL program. By leveraging Python, we can seamlessly transition from data manipulation in DataFrames to a fully functional database, streamlining our workflow and making the entire process faster and more accessible.

In [ ]:
# Create SQLite database and tables
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# Create tables employees_data
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department_id INTEGER NOT NULL,
    hire_date TEXT NOT NULL
);
""")

# Create tables departments_data
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    department_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);
""")

# Insert data into tables
employees_df.to_sql("employees", conn, if_exists="replace", index=False)
departments_df.to_sql("departments", conn, if_exists="replace", index=False)

# Verify data was inserted
print("Employees Table:\n")
print(pd.read_sql_query("SELECT * FROM employees", conn))

print("\nDepartments Table:\n")
print(pd.read_sql_query("SELECT * FROM departments", conn))




Employees Table:

   id     name  department_id   hire_date
0   1    Alice              1  2020-01-15
1   2      Bob              2  2019-07-22
2   3  Charlie              1  2021-03-30
3   4    David              3  2018-06-12
4   5      Eve              2  2022-11-03

Departments Table:

   department_id         name
0              1           HR
1              2  Engineering
2              3        Sales


In [ ]:
# employee_df.to_markdown()
# departments_df.to_markdown()

This is a great easy way to make or interact with database files. Good news is its not likely you will have to ever spin up a database in an entry level position. But having the skill for small things like this is valuable!

# Setting up query function

We will make a function that takes a string. So we will then be able to write queries in triple double quotes and pass then to our function.

In [ ]:
def query(query: str):
    return pd.read_sql(query, conn)

# Selecting all from a table.

testing function to see if it works correctly.

In [ ]:
all = """SELECT * FROM departments"""
query(all)

,department_id,name
0,1,HR
1,2,Engineering
2,3,Sales


In [ ]:
departments_df

,department_id,name
0,1,HR
1,2,Engineering
2,3,Sales


# Joins



In [ ]:
join = """
SELECT *
FROM employees
LEFT JOIN departments
ON employees.department_id = departments.department_id;
"""
query(join)

,id,name,department_id,hire_date,department_id,name
0,1,Alice,1,2020-01-15,1,HR
1,2,Bob,2,2019-07-22,2,Engineering
2,3,Charlie,1,2021-03-30,1,HR
3,4,David,3,2018-06-12,3,Sales
4,5,Eve,2,2022-11-03,2,Engineering


In [ ]:
join_df = employees_df.merge(departments_df, left_on="department_id", right_on="department_id", how="left")
join_df

,id,name_x,department_id,hire_date,name_y
0,1,Alice,1,2020-01-15,HR
1,2,Bob,2,2019-07-22,Engineering
2,3,Charlie,1,2021-03-30,HR
3,4,David,3,2018-06-12,Sales
4,5,Eve,2,2022-11-03,Engineering


Things to note:

- In the SQL query, both the `employees` and `departments` tables have a column named `name`, but the values they hold are different. In the `employees table`, name refers to the **employee's name**, while in the `departments table`, `name` refers to the **department name**. This discrepancy could lead to issues later on, which we will address at a later stage.

- In pandas, when merging or joining these tables, a unique suffix (_x and _y) is added to differentiate the name columns. This ensures that each column has a unique identifier, which makes it easier to work with the data.


Lets fix that below.

In [ ]:
join2 = """
SELECT employees.*, departments.name AS Department
FROM employees
LEFT JOIN departments
    ON employees.department_id = departments.department_id;
"""
query(join2)

,id,name,department_id,hire_date,Department
0,1,Alice,1,2020-01-15,HR
1,2,Bob,2,2019-07-22,Engineering
2,3,Charlie,1,2021-03-30,HR
3,4,David,3,2018-06-12,Sales
4,5,Eve,2,2022-11-03,Engineering


In [ ]:
# Perform the LEFT JOIN
joined_df = employees_df.merge(
    departments_df,
    how='left',
    on='department_id'
)

# Rename the 'name' columns
joined_df = joined_df.rename(columns={'name_y': 'Department', 'name_x': 'employee_name'})


joined_df

,id,employee_name,department_id,hire_date,Department
0,1,Alice,1,2020-01-15,HR
1,2,Bob,2,2019-07-22,Engineering
2,3,Charlie,1,2021-03-30,HR
3,4,David,3,2018-06-12,Sales
4,5,Eve,2,2022-11-03,Engineering




Earlier, we joined all of our data together using a `LEFT JOIN`, which is great for exploration—but it brought in more information than we actually need. Since we're working in pandas and the data is already local, it's not a big deal to pull everything and trim it down afterward.

<br>

However, this approach becomes much more important when working with SQL. In most real-world scenarios, especially with production databases hosted on remote servers or in the cloud, pulling unnecessary data can be costly—both in terms of time and money. That's why you'll rarely see a `SELECT *` in production environments. Instead, we only query the specific columns we need to keep our operations fast, efficient, and lightweight.



In [ ]:
slim_join = """
SELECT employees.name, employees.hire_date, departments.name AS Department
FROM employees
LEFT JOIN departments
    ON employees.department_id = departments.department_id;
"""

query(slim_join)

,name,hire_date,Department
0,Alice,2020-01-15,HR
1,Bob,2019-07-22,Engineering
2,Charlie,2021-03-30,HR
3,David,2018-06-12,Sales
4,Eve,2022-11-03,Engineering


In [ ]:
slim_df = employees_df.merge(
    departments_df,
    how='left',
    on='department_id'
)

# Rename the 'name' columns
slim_df = slim_df.rename(columns={'name_y': 'Department', 'name_x': 'name'})

# Select only the desired columns
slim_df = slim_df[['name', 'hire_date', 'Department']]

slim_df

,name,hire_date,Department
0,Alice,2020-01-15,HR
1,Bob,2019-07-22,Engineering
2,Charlie,2021-03-30,HR
3,David,2018-06-12,Sales
4,Eve,2022-11-03,Engineering


### What Are Aliases and How to Use Them
**What:**
<br>
Aliases are short names given to tables or columns in SQL queries to make them easier to read and manage. Instead of repeating long table names, you can use a short alias to simplify the query.

<br>

**Why It's Important:**
<br>
Without aliases, SQL queries can become long, repetitive, and hard to read—especially when joining multiple tables. This can lead to confusion and make debugging more difficult.

<br>

**How to Use Aliases Properly:**
<br>
Use the AS keyword (optional but recommended for clarity).

Choose short, meaningful aliases (e.g., u for users, o for orders).

Be consistent in your naming throughout the query.

### Example (Without Aliases):

In [ ]:
alias = """
SELECT employees.name, departments.name
FROM employees
INNER JOIN departments
ON employees.department_id = departments.department_id;
"""

query(alias)

,name,name
0,Alice,HR
1,Bob,Engineering
2,Charlie,HR
3,David,Sales
4,Eve,Engineering


### Example (With Aliases):

In [ ]:
Alias = """
SELECT e.name AS name, d.name AS name
FROM employees e
INNER JOIN departments d
ON e.department_id = d.department_id;
"""

query(Alias)

,name,name
0,Alice,HR
1,Bob,Engineering
2,Charlie,HR
3,David,Sales
4,Eve,Engineering


### WHERE vs HAVING

New users often confuse WHERE (filters rows before grouping) with HAVING (filters grouped data).

#### WHERE example


In [ ]:
where_vs_having = """
SELECT id, name
FROM employees
WHERE id >= 3;
"""

query(where_vs_having)

,id,name
0,3,Charlie
1,4,David
2,5,Eve


The issue is using `HAVING` without a `GROUP BY` clause. The `HAVING` clause is used to filter aggregated results after grouping, whereas `WHERE` is used to filter rows before any aggregation happens.



The SQL engine doesn't know how to handle department_id in the HAVING clause because you haven't grouped by it.

In [ ]:
where_vs_having = """
SELECT department_id, COUNT(*) AS total_employees
FROM employees
GROUP BY department_id
HAVING department_id = 1;
"""

query(where_vs_having)

,department_id,total_employees
0,1,2


| Keyword     | Description                                                                 |
|-------------|-----------------------------------------------------------------------------|
| SELECT      | Used to specify the columns to retrieve from a table.                      |
| FROM        | Specifies the table(s) to retrieve the data from.                          |
| WHERE       | Filters rows **before** any grouping or aggregation.                      |
| GROUP BY    | Groups rows that have the same values in specified columns.               |
| HAVING      | Filters groups **after** `GROUP BY` and aggregation.                      |
| ORDER BY    | Sorts the result set by one or more columns (ASC or DESC).                |
| INSERT INTO | Adds new rows to a table.                                                  |
| VALUES      | Specifies the values to insert into the table.                             |
| UPDATE      | Modifies existing records in a table.                                      |
| SET         | Used with `UPDATE` to specify column values to change.                     |
| DELETE      | Removes rows from a table.                                                 |
| JOIN        | Combines rows from two or more tables based on a related column.          |
| INNER JOIN  | Returns records that have matching values in both tables.                 |
| LEFT JOIN   | Returns all records from the left table, and matched records from the right. |
| RIGHT JOIN  | Returns all records from the right table, and matched records from the left. |
| FULL JOIN   | Returns all records when there is a match in either left or right table.  |
| ON          | Specifies the condition for the `JOIN`.                                   |
| AS          | Renames a column or table with an alias.                                  |
| DISTINCT    | Returns only unique values.                                                |
| LIMIT       | Restricts the number of rows returned.                                     |
| OFFSET      | Skips a specified number of rows before beginning to return rows.          |
| IN          | Checks if a value matches any value in a list or subquery.                 |
| BETWEEN     | Checks if a value is within a given range.                                 |
| LIKE        | Performs pattern matching using wildcards (`%` or `_`).                   |
| IS NULL     | Checks whether a column has a NULL value.                                  |
| UNION       | Combines results of two or more `SELECT` statements (removes duplicates).  |
| UNION ALL   | Same as `UNION`, but includes duplicates.                                  |


# SQL Cheatsheet

A quick-reference guide to essential SQL keywords and concepts.

*Note:* SQL syntax may vary slightly between databases like MySQL, PostgreSQL, SQL Server, and SQLite.

---

## Data Querying

| Keyword     | Description                                         |
|-------------|-----------------------------------------------------|
| `SELECT`    | Retrieves data from a database.                     |
| `FROM`      | Specifies the table(s) to query data from.          |
| `WHERE`     | Filters rows based on a condition.                  |
| `GROUP BY`  | Groups rows sharing a property (used with aggregates). |
| `HAVING`    | Filters groups after `GROUP BY`.                    |
| `ORDER BY`  | Sorts the results in ascending (`ASC`) or descending (`DESC`) order. |
| `LIMIT`     | Restricts the number of returned rows.              |
| `OFFSET`    | Skips a specified number of rows.                   |
| `DISTINCT`  | Returns only unique values.                         |

---

## Conditions & Operators

| Operator     | Description                                           |
|--------------|-------------------------------------------------------|
| `=`          | Equals                                                |
| `!=`, `<>`   | Not equal                                             |
| `<`, `>`, `<=`, `>=` | Comparison operators                        |
| `BETWEEN`    | Checks if a value is within a range                  |
| `IN`         | Checks if a value exists in a list                   |
| `LIKE`       | Pattern match using `%` (wildcard) and `_` (single char) |
| `IS NULL`    | Checks for null values                               |
| `AND`, `OR`, `NOT` | Logical operators                            |

---

## Joins

| Join Type      | Description                                                      |
|----------------|------------------------------------------------------------------|
| `INNER JOIN`   | Returns only matching rows in both tables.                      |
| `LEFT JOIN`    | All rows from the left table + matches from the right.          |
| `RIGHT JOIN`   | All rows from the right table + matches from the left.          |
| `FULL JOIN`    | All rows when there's a match in either table.                  |
| `ON`           | Defines the join condition.                                      |

---

## Data Aggregation

| Function     | Description                    |
|--------------|--------------------------------|
| `COUNT()`    | Counts rows                    |
| `SUM()`      | Adds values                    |
| `AVG()`      | Calculates average             |
| `MIN()`      | Finds the minimum              |
| `MAX()`      | Finds the maximum              |
| `GROUP BY`   | Groups rows for aggregation    |
| `HAVING`     | Filters aggregated groups      |

---

## Data Manipulation (DML)

| Command        | Description                              |
|----------------|------------------------------------------|
| `INSERT INTO`  | Adds new rows to a table.                |
| `VALUES`       | Specifies values to insert.              |
| `UPDATE`       | Modifies existing data.                  |
| `SET`          | Defines columns and new values for update.|
| `DELETE`       | Removes rows from a table.               |

---

## Data Definition (DDL)

| Command        | Description                              |
|----------------|------------------------------------------|
| `CREATE TABLE` | Defines a new table.                     |
| `DROP TABLE`   | Deletes a table.                         |
| `ALTER TABLE`  | Modifies table structure.                |
| `TRUNCATE`     | Deletes all data but keeps table.        |
| `RENAME`       | Renames a table or column.               |

---

## Control Flow & Logic

| Keyword  | Description                                |
|----------|--------------------------------------------|
| `CASE`   | Conditional logic in queries.              |
| `WHEN`   | Used inside a `CASE` to define conditions. |
| `THEN`   | Returns value if `WHEN` is true.           |
| `ELSE`   | Default value if no `WHEN` condition is met.|
| `END`    | Closes the `CASE` statement.               |

---

## Advanced: Window Functions

| Keyword        | Description                                        |
|----------------|----------------------------------------------------|
| `OVER`         | Applies a window function.                        |
| `PARTITION BY` | Divides result set into partitions.               |
| `ROW_NUMBER()` | Assigns unique row numbers per partition.         |
| `RANK()`       | Ranks rows with gaps for ties.                    |
| `DENSE_RANK()` | Ranks rows without gaps for ties.                 |

---

## Transactions

| Keyword    | Description                             |
|------------|-----------------------------------------|
| `BEGIN`    | Starts a new transaction.               |
| `COMMIT`   | Saves the changes made in the transaction.|
| `ROLLBACK` | Undoes the changes in the transaction.  |
| `SAVEPOINT`| Creates a savepoint in the transaction. |

---

## Common Table Expressions (CTE)

| Keyword | Description                         |
|---------|-------------------------------------|
| `WITH`  | Declares a temporary named result set (like a subquery). |

---

## Set Operations

| Operator      | Description                                            |
|---------------|--------------------------------------------------------|
| `UNION`       | Combines results of two queries (removes duplicates).  |
| `UNION ALL`   | Combines results, includes duplicates.                 |
| `INTERSECT`   | Returns rows common to both queries.                   |
| `EXCEPT`      | Returns rows from the first query not in the second.  |

---

## Extras

| Keyword   | Description                             |
|-----------|-----------------------------------------|
| `AS`      | Renames a column or table using an alias.|
| `EXISTS`  | Checks if a subquery returns any rows.   |
| `ALL`     | Compares a value to all values in a list.|
| `ANY`     | Compares a value to any value in a list. |

---

